# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [31]:
# Dependencies and Setup
# !pip install cartopy geoviews pyproj
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path
import json

# Import API key
from api_keys import geoapify_key

In [32]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,edinburgh of the seven seas,-37.0676,-12.3116,15.14,85,100,15.40,SH,1693047401
1,1,thompson,55.7435,-97.8558,7.09,93,0,1.54,CA,1693047226
2,2,adamstown,-25.0660,-130.1015,18.69,64,82,6.82,PN,1693047167
3,3,bredasdorp,-34.5322,20.0403,13.48,49,66,6.63,ZA,1693047414
4,4,albany,42.6001,-73.9662,19.77,94,49,2.07,US,1693047211


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [33]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points("Lng","Lat",geo = True,tiles = "OSM",size = "Humidity",scale =0.8,color = "City")

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [72]:
# Narrow down cities that fit criteria and drop any results with null values
max_Temp=25
min_temp=20
Countries = ["US", "AU", "CA"]
filtered_cities=city_data_df[(city_data_df['Max Temp'] <= max_Temp) &(city_data_df['Max Temp'] >= min_temp)&(city_data_df['Country'].isin(Countries))]

# Drop any rows with null values
filtered_cities = filtered_cities.dropna()

# Display sample data
filtered_cities

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
10,10,bethel,41.3712,-73.4140,22.20,89,100,0.00,US,1693047417
29,29,camden,39.7835,-74.9663,23.14,96,49,0.45,US,1693047875
43,43,cannonvale,-20.2833,148.7000,20.66,74,0,7.04,AU,1693047880
47,47,racine,42.7495,-88.0751,20.83,96,100,2.06,US,1693047881
67,67,holualoa,19.6228,-155.9522,20.28,85,57,2.24,US,1693047448
84,84,hawaiian paradise park,19.5933,-154.9731,22.16,95,100,2.06,US,1693047391
130,130,laguna,38.4210,-121.4238,20.52,82,0,2.06,US,1693047638
135,135,palmer,42.1584,-72.3287,22.01,83,40,0.00,US,1693047914
136,136,portland,45.5234,-122.6762,20.41,79,0,1.54,US,1693047860
149,149,westport,41.1415,-73.3579,22.90,92,75,0.00,US,1693047920


### Step 3: Create a new DataFrame called `hotel_df`.

In [74]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
columns_to_copy = ['City', 'Country', 'Lat', 'Lng', 'Humidity']
hotel_df = filtered_cities[columns_to_copy].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
10,bethel,US,41.3712,-73.4140,89,
29,camden,US,39.7835,-74.9663,96,
43,cannonvale,AU,-20.2833,148.7000,74,
47,racine,US,42.7495,-88.0751,96,
67,holualoa,US,19.6228,-155.9522,85,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

Starting hotel search
bethel - nearest hotel: Hotel Pestana Plaza Mayor
camden - nearest hotel: Hotel Pestana Plaza Mayor
cannonvale - nearest hotel: Hotel Pestana Plaza Mayor
racine - nearest hotel: Hotel Pestana Plaza Mayor
holualoa - nearest hotel: Hotel Pestana Plaza Mayor
hawaiian paradise park - nearest hotel: Hotel Pestana Plaza Mayor
laguna - nearest hotel: Hotel Pestana Plaza Mayor
palmer - nearest hotel: Hotel Pestana Plaza Mayor
portland - nearest hotel: Hotel Pestana Plaza Mayor
westport - nearest hotel: Hotel Pestana Plaza Mayor
perry - nearest hotel: Hotel Pestana Plaza Mayor
hilo - nearest hotel: Hotel Pestana Plaza Mayor
fort bragg - nearest hotel: Hotel Pestana Plaza Mayor
haiku-pauwela - nearest hotel: Hotel Pestana Plaza Mayor
hamilton - nearest hotel: Hotel Pestana Plaza Mayor
carnarvon - nearest hotel: Hotel Pestana Plaza Mayor
the dalles - nearest hotel: Hotel Pestana Plaza Mayor
yarmouth - nearest hotel: Hotel Pestana Plaza Mayor
retreat - nearest hotel: Hotel Pe

,City,Country,Lat,Lng,Humidity,Hotel Name
10,bethel,US,41.3712,-73.4140,89,Hotel Pestana Plaza Mayor
29,camden,US,39.7835,-74.9663,96,Hotel Pestana Plaza Mayor
43,cannonvale,AU,-20.2833,148.7000,74,Hotel Pestana Plaza Mayor
47,racine,US,42.7495,-88.0751,96,Hotel Pestana Plaza Mayor
67,holualoa,US,19.6228,-155.9522,85,Hotel Pestana Plaza Mayor
84,hawaiian paradise park,US,19.5933,-154.9731,95,Hotel Pestana Plaza Mayor
130,laguna,US,38.4210,-121.4238,82,Hotel Pestana Plaza Mayor
135,palmer,US,42.1584,-72.3287,83,Hotel Pestana Plaza Mayor
136,portland,US,45.5234,-122.6762,79,Hotel Pestana Plaza Mayor
149,westport,US,41.1415,-73.3579,92,Hotel Pestana Plaza Mayor


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)